# 演習1：タスク指向システム

概要
* レストラン案内タスクを行う音声対話システムを作成
* 音声認識と音声合成は既存のサービスを使用
* 言語理解・対話管理（応答生成を含む）については一から実装

目的
* pythonを用いてタスク指向の音声対話システムを順番に一通り実装してみることで、理論と実装のギャップを埋める

## 必要なライブラリのインストール

はじめに、必要となるライブラリをインストールします。ここではpipを使ってインストールしますが、condaでも同様のライブラリをインストールすることができます。ご自身の環境に合わせてインストール方法を適宜変更してください。

また、以下の「!」で始まるコマンドは、ターミナル（コマンドプロンプト）で「!」を除いた部分を実行するこを意味します。「!」をつけることで、通常のコマンドをJupyter Notebook経由で実行できるようになります。

In [1]:
# 音声認識
# Macの場合brew install portaudioを実行してからpyaudioをインストールする
! pip install pyaudio
! pip install SpeechRecognition

# 音声合成
! pip install gTTS
! pip install pygame

## Step 1: 音声認識

speech_recognition というライブラリを使用します。このライブラリの内部ではGoogleの音声認識サービスが呼び出されています。したがって、インターネット接続が必要となります。

In [8]:
# ライブラリのインポート 
import speech_recognition as sr

# ライブラリを初期化
r = sr.Recognizer()

# パラメータの設定
# ユーザが発話を止めてから何秒間待つか
# この値が大きいとシステムの反応が遅くなる
# 逆に小さくすると、反応は早くなるが、ユーザの発話が終了していないのに、システムが割り込んでしまうリスクが高くなる
r.pause_threshold = 0.5

# デフォルトに設定されたマイクから音声入力を受け付ける
# もしうまく動かない場合はマイク設定を確認してください
with sr.Microphone() as source:
    r.adjust_for_ambient_noise(source) # 背景雑音へ適応する（１秒間）
    print("どうぞ話してください >> ")
    audio = r.listen(source)

print("認識開始")

# Google Web Speech APIを用いて音声認識を行う
# 認識言語を設定することができ、ここでは日本語（"ja-JP"）とする
try:
    print("認識結果:" + r.recognize_google(audio, language="ja-JP"))
except sr.UnknownValueError:
    print("認識エラー")
except sr.RequestError as e:
    print("サーバエラー {0}".format(e))

どうぞ話してください >> 
認識開始
認識結果:えますきっと クラーケンというですね ソフトなんかを使っていたりします で これは です ね まふりーもあるんですけれども基本的には29$3000円とかそういったところが ですね かかってくる ソフトなんですが 無理のものもあったりするのでこういったものを使ってキットを使っていくというやり方が一般的ですで今回はまこういったですね GI ソフト 使うのもいいんですけれども Microsoft が作っている ビジュアルスタジオコード というですね エディター ソフトがあるんですが これが ですね ギットに対応していますので このエディター ソフトを使ってキットを操作していくというやり方について紹介していきたいと思います


## Step 2: 音声合成

gttsというライブラリを使用します。このライブラリでは、Googleの音声合成サービスを利用して、音声ファイルを作成します。そして、作成された音声ファイルをpygameというライブラリを用いて再生します。

In [16]:
# ライブラリのインポート
from gtts import gTTS
import pygame

# 合成したいテキストを設定
text = "隣の客はよく柿食う客だ"

# 音声合成を実行
# 生成する言語を指定でき、ここでは日本語（"ja"）に設定
print('合成開始: ' + text)
speech = gTTS(text=text, lang="ja")
print('合成完了')

# 生成した音声ファイルをmp3ファイルとして一時保存
# この一連の処理を複数回実行する際に最後の解放処理を実行しないと（例えば、途中で処理を中断した場合）
# エラーになることがあります。その場合はJupyter Notebookを再起動してください。
try:
    speech.save("./data/test.mp3")
except Exception as e:
    print('ファイル保存エラー')
    
# 保存した音声ファイルを再生
pygame.mixer.init()
pygame.mixer.music.load("./data/test.mp3")
pygame.mixer.music.play()

print('再生中')

# 再生が完了するまて待機
while pygame.mixer.music.get_busy():
    pygame.time.Clock().tick(10)

# ファイルを解放する
pygame.mixer.music.stop()
pygame.mixer.quit()

print('再生終了')

合成開始: 隣の客はよく柿食う客だ
合成完了
再生中
再生終了


## Step 3: 言語理解

正規表現を使用してルールベースのスロット値抽出を実装します。


はじめに、抽出したいスロットについて、具体的な値とスロット名を定義します。

In [17]:
#
# 抽出したいスロットを正規表現として列挙
# 正規表現：特定のパターンを記述するための文字列で、文字列内の特定の部分を検索、マッチング、抽出、置換などを行う。
#

# 例１：地名を OR 表現で列挙
place = '(京都|今出川)'

# 例２：料理のジャンルを OR 表現で列挙
genre = '(ラーメン|イタリアン|そば)'

# 例３：店名
name = '(味亭|割烹井上)'

# 例４：予算
budget = '(1000円以下|2000円以下|3000円以下)'


#
# 次に、上記の各スロットにスロット名を付与し、リストとして表現
# 例えば、地名のスロットは place というスロット名とする
#

def_slot = [
    ['place', place],
    ['genre', genre],
    ['name', name],
    ['budget', budget]
]

# 作成したリストの情報を表示
for elem_slot in def_slot:
    print(elem_slot[0] + ' : ' + elem_slot[1])

place : (京都|今出川)
genre : (ラーメン|イタリアン|そば)
name : (味亭|割烹井上)
budget : (1000円以下|2000円以下|3000円以下)


次に、正規表現のライブラリを用いて入力文（発話文）とマッチングする関数を作成します。

In [18]:
# 正規表現のライブラリを読み込む
import re

#
# 入力文と各スロットをマッチングする関数
#
# 引数：
#       input_sentence 入力発話文
#       def_slot スロットの定義データ
# 戻値：
#       マッチしたスロットの名前と値を含む辞書のリスト
#
def matching_slu(input_sentence: str, list_def_slot: list) -> list:
    
    # マッチしたスロットの名前と値を格納するリスト
    result_all = []
    
    # 各スロットの情報をfor文で取り出す
    for def_slot in list_def_slot:
        
        # re.findallを用いて、入力発話文がそのスロットを含むかマッチング
        result = re.findall(def_slot[1], input_sentence)
        
        # 含まれていたら対応するスロットの名前と値を抽出する
        if len(result) >= 1:
            
            # 複数マッチする場合に対応
            for r in result:
                
                # スロット名
                slot_name = def_slot[0]
                
                # スロット値
                slot_value = r
                
                result_all.append({'slot_name': slot_name, 'slot_value': slot_value})
    
    return result_all

作成した関数をテストします。ここでは、テスト用のデータを読み込んでみて、それを作成した関数に入力します。

In [19]:
# テスト入力データを設定する
list_input_data = [
    "京都のおいしいラーメンを教えてください",
    "今出川の近くでイタリアンはありますか",
    "味亭の営業時間を教えて",
    "割烹井上は何時から開いていますか",
    "近くにおいしいそば屋はありますか",
    "京都で2000円以下のおいしいイタリアンはありますか"
]

# 入力データを作成した関数を用いてマッチするか判定する
for input_data in list_input_data:
    
    print('入力：' + input_data)
    
    # 判定する関数を呼び出す
    slu_result = matching_slu(input_data, def_slot)
    
    # 戻り値に応じて抽出した要素を表示
    if len(slu_result) == 0:
        print('抽出スロットなし')
    
    else:
        for s in slu_result:
            print('{}: {}'.format(s['slot_name'], s['slot_value']))
    
    print()

入力：京都のおいしいラーメンを教えてください
place: 京都
genre: ラーメン

入力：今出川の近くでイタリアンはありますか
place: 今出川
genre: イタリアン

入力：味亭の営業時間を教えて
name: 味亭

入力：割烹井上は何時から開いていますか
name: 割烹井上

入力：近くにおいしいそば屋はありますか
genre: そば

入力：京都で2000円以下のおいしいイタリアンはありますか
place: 京都
genre: イタリアン
budget: 2000円以下



## Step 4: 対話管理

下図のような状態遷移モデルを実装します。
このような状態遷移を実装するには、まずは状態と遷移の情報を記述し、それをもとにユーザ発話の情報に応じて状態を制御する部分を作成します。
下図において、状態はシステム発話に、遷移はユーザ発話にそれぞれ対応します。

<img src="./img/automaton.png" style="width: 800px;"/>

まずは、状態を定義します。

状態番号は下記とします。
- 図中の左側について、上から順に０～３
- 図中の右側について、上から順に４～６

In [20]:
# 状態を定義
# 状態番号、対応するシステム発話
STATES = [
    [0, 'こんにちは。京都レストラン案内です。どの地域のレストランをお探しですか。'],
    [1, 'どのような料理がお好みですか。'],
    [2, 'ご予算はおいくらぐらいですか。'],
    [3, '検索します。'],
    [4, '地域名を「京都駅近辺」のようにおっしゃってください。'],
    [5, '和食・洋食・中華・ファストフードからお選びください。'],
    [6, '予算を「3000円以下」のようにおっしゃってください。']    
]

# 開始状態の状態番号
START_STATE = 0

# 終了状態の状態番号
END_STATE = 3

次に、遷移を定義します。遷移元と遷移先の状態番号、条件となるユーザ発話の情報で構成します。ユーザ発話の情報は、ここでは言語理解で設計したスロット名（place や genre など）にします。また、後で実装しますが、スロット値は変数として保存しておきます。また、遷移条件の「None」はそれより上の条件にマッチしなかった場合の「それ以外の入力」に相当します。

In [21]:
# 遷移を定義
# 遷移元状態番号、遷移先状態番号、遷移条件（スロット名）
TRANS = [
    [0, 1, 'place'],
    [0, 4, None],
    [1, 2, 'genre'],
    [1, 5, None],
    [2, 3, 'budget'],
    [2, 6, None],
    [4, 1, 'place'],
    [4, 4, None],
    [5, 2, 'genre'],
    [5, 5, None],
    [6, 3, 'budget'],
    [6, 6, None]
]

では、定義した状態と遷移に基づいて対話を制御します。内部変数として、現在の状態番号を保持し、入力であるユーザ発話に応じてシステム発話を出力し、状態を遷移させます。この処理では内部変数を保持する必要があるので、全体をクラス化します。

In [22]:
# 対話管理を行うクラス
class DialogManager:

    current_state: int              # 現在の状態
    context_user_utterance: dict    # 遷移条件にマッチしたユーザ発話の情報を保持する辞書（対話によって得られた情報）

    # コンストラクタ（クラスのインスタンスを初期化したときに呼ばれる特殊なメソッド）
    def __init__(self):

        # 内部状態などを初期化
        self.reset()

    # 入力であるユーザ発話に応じてシステム発話を出力し、内部状態を遷移させる
    # ただし、ユーザ発話の情報は「意図、スロット名、スロット値」のlistとする
    def enter(self, user_utterance):
        
        # フレーム名に対して行う
        # 最初の0番目のindexは1発話に対して複数のスロットが抽出された場合に対応するため
        # ここでは1発話につき１つのフレームしか含まれないという前提
        if len(user_utterance) > 0:
            input_slot_name = user_utterance[0]['slot_name']
            input_slot_value = user_utterance[0]['slot_value']
        else:
            # スロットが抽出されなかった場合
            input_slot_name = None
            input_slot_value = None
        
        system_utterance = ""
        
        # 現在の状態からの遷移に対して入力がマッチするか検索
        for trans in TRANS:
            
            # 条件の遷移元が現在の状態か
            if trans[0] == self.current_state:
                
                # 無条件に遷移
                if trans[2] is None:
                    self.current_state = trans[1]
                    system_utterance = self.get_system_utterance()
                    break
                
                # 条件にマッチすれば遷移
                if trans[2] == input_slot_name:

                    # 遷移条件にマッチしたユーザ発話の情報を保持する辞書に格納
                    self.context_user_utterance[input_slot_name] = input_slot_value
                    
                    self.current_state = trans[1]
                    system_utterance = self.get_system_utterance()
                    break
        
        return system_utterance

    # 初期状態にリセットする
    def reset(self):

        # 内部状態を初期状態にする
        self.current_state = START_STATE

        # 遷移条件にマッチしたユーザ発話の情報を保持する辞書を初期化
        self.context_user_utterance = {}


    # 指定された状態に対応するシステムの発話を取得
    def get_system_utterance(self):
        
        utterance = ""
        
        for state_ in STATES:
            if self.current_state == state_[0]:
                utterance = state_[1]
        
        return utterance

実装した対話管理をテストしましょう。

In [23]:
# 初期化
dm = DialogManager()
dm.reset()

# 初期状態の発話を表示
print("システム発話 : " + dm.get_system_utterance())

システム発話 : こんにちは。京都レストラン案内です。どの地域のレストランをお探しですか。


In [24]:
# ユーザ発話を設定
user_utterance = [{'slot_name': 'place', 'slot_value': '京都駅周辺'}]
print('ユーザ発話')
print(user_utterance)

print()

# 次のシステム発話を表示
print('システム発話')
print(dm.enter(user_utterance))

ユーザ発話
[{'slot_name': 'place', 'slot_value': '京都駅周辺'}]

システム発話
どのような料理がお好みですか。


In [25]:
# 想定外の発話を入力してみる

# ユーザ発話を設定
user_utterance = [{'slot_name': 'place', 'slot_value': '新宿'}]
print('ユーザ発話')
print(user_utterance)
print()

# 次のシステム発話を表示
print('システム発話')
print(dm.enter(user_utterance))

ユーザ発話
[{'slot_name': 'place', 'slot_value': '新宿'}]

システム発話
和食・洋食・中華・ファストフードからお選びください。


In [26]:
# ユーザ発話を設定
user_utterance = [{'slot_name': 'genre', 'slot_value': '和食'}]
print('ユーザ発話')
print(user_utterance)
print()

# 次のシステム発話を表示
print('システム発話')
print(dm.enter(user_utterance))

ユーザ発話
[{'slot_name': 'genre', 'slot_value': '和食'}]

システム発話
ご予算はおいくらぐらいですか。


In [27]:
# ユーザ発話を設定
user_utterance = [{'slot_name': 'budget', 'slot_value': '3000円以下'}]
print('ユーザ発話')
print(user_utterance)
print()

# 次のシステム発話を表示
print('システム発話')
print(dm.enter(user_utterance))

# 得られた一連のユーザ発話の情報を表示
print('対話によって得られた情報')
print('場所：' + dm.context_user_utterance['place'])
print('ジャンル：' + dm.context_user_utterance['genre'])
print('予算：' + dm.context_user_utterance['budget'])

ユーザ発話
[{'slot_name': 'budget', 'slot_value': '3000円以下'}]

システム発話
検索します。
対話によって得られた情報
場所：京都駅周辺
ジャンル：和食
予算：3000円以下


## Step 5: システム統合

ここまで実装してきた各モジュールを統合し、音声対話システムとして動作するようにします。

まず、はじめに音声認識と音声合成のモジュールを呼び出しやすく関数化しておきます。

In [28]:
# 音声認識を関数化
def get_asr():
    
    r = sr.Recognizer()
    r.pause_threshold = 0.5
    
    with sr.Microphone() as source:
        r.adjust_for_ambient_noise(source) # 背景雑音へ適応する（１秒間）
        print("どうぞ話してください >> ")
        audio = r.listen(source)
    
    try:
        result = r.recognize_google(audio, language="ja-JP")
    except sr.UnknownValueError:
        result = ""
    except sr.RequestError as e:
        result = ""
    
    return result

# 音声合成を関数化
def play_tts(text):
    
    speech = gTTS(text=text, lang="ja")

    try:
        speech.save("./data/test.mp3")
    except Exception as e:
        print('ファイル保存エラー')
    
    pygame.mixer.init()
    pygame.mixer.music.load("./data/test.mp3")
    pygame.mixer.music.play()

    while pygame.mixer.music.get_busy():
        pygame.time.Clock().tick(10)

    pygame.mixer.music.stop()
    pygame.mixer.quit()

そして、言語理解と対話管理を順に呼び出すことで対話を進めます。

In [31]:
# 対話管理を初期化
dm = DialogManager()

# 初期状態の発話
system_utterance = dm.get_system_utterance()
print("システム： " + system_utterance)

# 音声合成
play_tts(system_utterance)

# 対話が終了状態に移るまで対話を続ける
while dm.current_state != END_STATE:
    
    # 音声入力＆音声認識
    result_asr_utterance = get_asr()
    print("ユーザ： " + result_asr_utterance)
    
    # 言語理解
    result_slu = matching_slu(result_asr_utterance, def_slot)
    print(result_slu)
    
    # 対話管理へ入力
    system_utterance = dm.enter(result_slu)
    print("システム： " + system_utterance)
    play_tts(system_utterance)
    
    print()

# 対話終了
print("対話終了")

# 得られた一連のユーザ発話の情報を表示
print('対話によって得られた情報')
print('場所：' + dm.context_user_utterance['place'])
print('ジャンル：' + dm.context_user_utterance['genre'])
print('予算：' + dm.context_user_utterance['budget'])

システム： こんにちは。京都レストラン案内です。どの地域のレストランをお探しですか。
どうぞ話してください >> 
ユーザ： 最近では 今出川です
[{'slot_name': 'place', 'slot_value': '今出川'}]
システム： どのような料理がお好みですか。

どうぞ話してください >> 
ユーザ： 僕が好きなのは イタリアンです
[{'slot_name': 'genre', 'slot_value': 'イタリアン'}]
システム： ご予算はおいくらぐらいですか。

どうぞ話してください >> 
ユーザ： は
[]
システム： 予算を「3000円以下」のようにおっしゃってください。

どうぞ話してください >> 
ユーザ： 金欠なので1000円以下で
[{'slot_name': 'budget', 'slot_value': '1000円以下'}]
システム： 検索します。

対話終了
対話によって得られた情報
場所：今出川
ジャンル：イタリアン
予算：1000円以下
